In [2]:
from datetime import datetime
import plotly.graph_objects as pg
import plotly.express as px
import plotly.io as pio
import pandas as pd
import locale
locale.setlocale(locale.LC_TIME, "")


from cashd import db

CORES = ["#478eff", "gray"]

Este notebook será usado para experimentar e desenvolver visualizações de maneira mais eficiente.

In [3]:
periodo = "sem"
n = 10

tbl = db.saldos_transac_periodo(periodo=periodo, n=n)
tbl["Data"].apply(lambda x: datetime.strptime(x+"-0", "%Y-%W-%w"))

63   2024-03-24
64   2024-03-31
65   2024-04-07
66   2024-04-14
67   2024-04-21
68   2024-04-28
69   2024-05-05
70   2024-05-12
71   2024-05-19
72   2024-05-26
Name: Data, dtype: datetime64[ns]

In [10]:
### balancos_por_periodo
periodo = "dia"
n = 30

tbl = db.saldos_transac_periodo(periodo=periodo, n=n)
if periodo == "sem":
    tbl["Data"] = tbl["Data"].apply(lambda x: datetime.strptime(x+"-0", "%Y-%W-%w"))
tbl["Data"] = pd.to_datetime(tbl.Data)
tbl["SomasDisplay"] = tbl["Somas"]\
    .apply(lambda x: f"{x:_.2f}".replace(".", ",").replace("_", " "))
tbl["AbatDisplay"] = tbl["Abatimentos"]\
    .apply(lambda x: f"{x:_.2f}".replace(".", ",").replace("_", " "))

datestr = "%B de %Y"
if periodo == "dia":
    datestr = "%d de %B de %Y"
elif periodo == "sem":
    datestr = "%Y, semana %W"

layout = pg.Layout(
    margin=dict(l=0, r=0, t=0, b=0),
    template="none",
    showlegend=False,
    hovermode="x unified",
    xaxis=dict(
        tickmode="array",
        tickvals=[i for i in tbl["Data"]],
        ticktext=[i.strftime(datestr) for i in tbl["Data"]]
    )
)

fig = pg.Figure(layout=layout)
fig.add_trace(pg.Bar(
    x=tbl["Data"], y=tbl["Somas"], name="Somas",
    customdata=tbl[["SomasDisplay"]],
    hovertemplate="<b>R$ %{customdata[0]}</b>",
    offsetgroup=0,
    marker=dict(color=CORES[1])
))
fig.add_trace(pg.Bar(
    x=tbl["Data"], y=tbl["Abatimentos"], name="Abatimentos",
    customdata=tbl[["AbatDisplay"]],
    hovertemplate="<b>R$ %{customdata[0]}</b>",
    offsetgroup=0,
    marker=dict(color=CORES[0])
))
fig.show(config={"displayModeBar": False})

In [17]:
### saldos_por_periodo
periodo = "sem"
n = 30

tbl = db.saldos_transac_periodo(periodo=periodo)
if periodo == "sem":
    tbl["Data"] = tbl["Data"].apply(lambda x: datetime.strptime(x+"-0", "%Y-%W-%w"))
tbl["Data"] = pd.to_datetime(tbl.Data)

tbl["SaldoAcum"] = (tbl["Somas"] + tbl["Abatimentos"]).cumsum()
tbl["SaldoAcumDisplay"] = tbl["SaldoAcum"]\
    .apply(lambda x: f"{x:_.2f}".replace(".", ",").replace("_", " "))

if n:
    tbl = tbl.tail(n)

datestr = "%B de %Y"
if periodo == "dia":
    datestr = "%d de %B de %Y"
elif periodo == "sem":
    datestr = "%Y, semana %W"

layout = pg.Layout(
    margin=dict(l=0, r=0, t=0, b=0),
    template="none",
    showlegend=False,
    hovermode="x unified",
    xaxis=dict(
        tickmode="array",
        tickvals=[i for i in tbl["Data"]],
        ticktext=[i.strftime(datestr) for i in tbl["Data"]]
    )
)

fig = pg.Figure(layout=layout)
fig.add_trace(pg.Scatter(
    x=tbl["Data"], y=tbl["SaldoAcum"], name="Saldo", 
    mode="lines+markers", 
    customdata=tbl[["SaldoAcumDisplay"]],
    hovertemplate="<b>R$ %{customdata[0]}</b>",
    offsetgroup=0,
    marker=dict(color=CORES[0])
))
fig.update_xaxes(showgrid=False)
fig.show(config={"displayModeBar": False})